<a href="https://colab.research.google.com/github/Billy-freespace/CUDA/blob/main/ejemCUDA01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!sudo ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!sudo ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

61 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2021-01-18 13:33:06--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?SRiS1bcBD4xCce9epy_HJ1FUNUReo2w9DZZ_pZXZCnNdB92nSKIPT8Xm-F3KqAQRHDD9yZBCbyh_ktb0ughiIi7-eS6mDUan3BBfIxj47jflRuM4Vh5tJZdST-CnXReKVIlNIUQx5XGgAXtYrUtBha4HOomI59PEIgeMMwe4BG19teJd436Dbb7RqocjrG-pvXBWxvzfphdZjcJko8sCkvfqCg [following]
--2021-01-18 13:33:07--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?SRiS1bcBD4xCce9epy_HJ1FUNURe

In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ymkv02d2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ymkv02d2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=220a4f8f61dbf4595a007fcce3f27bb1a4f150bfd49d3a6cddfebcfac727108b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ldwr5b5/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include <iostream>

int main() {
    std::cout << "Hello world\n";
    return 0;
}

Hello world



In [ ]:
%%cu

#include <iostream>
#include <algorithm>

using namespace std;

#define N 1024
#define RADIUS 3
#define BLOCK_SIZE 16

__global__ void stencil_1d(int *in, int *out) {
  __shared__ int temp[BLOCK_SIZE + 2 * RADIUS];
  int gindex = threadIdx.x + blockIdx.x * blockDim.x;
  int lindex = threadIdx.x + RADIUS;

  // device code
  // Read input elements into shared memory
  temp[lindex] = in[gindex];
  if (threadIdx.x < RADIUS) {
    temp[lindex - RADIUS] = in[gindex - RADIUS];
    temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
  }

  // Synchronize (ensure all the data is available)
  __syncthreads();
  
  // Apply the stencil
  int result = 0;
  for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
    result += temp[lindex + offset];

  // Store the result
  out[gindex] = result;
}

void fill_ints(int *x, int n) {
  fill_n(x, n, 1);
}

// serial function
int main(void) {
  int *in, *out; // host copies of a, b, c
  int *d_in, *d_out; // device copies of a, b, c  
  int size = (N + 2*RADIUS) * sizeof(int);

  // Alloc space for host copies and setup values
  in = (int *)malloc(size); fill_ints(in, N + 2*RADIUS);
  out = (int *)malloc(size); fill_ints(out, N + 2*RADIUS);

  // Alloc space for device copies
  cudaMalloc((void **)&d_in, size);
  cudaMalloc((void **)&d_out, size);
  
  // host code
  // Copy to device
  cudaMemcpy(d_in, in, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_out, out, size, cudaMemcpyHostToDevice);

  // Launch stencil_1d() kernel on GPU
  stencil_1d<<<N/BLOCK_SIZE,BLOCK_SIZE>>>(d_in + RADIUS, d_out + RADIUS);

  // Copy result back to host
  cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);

  // Cleanup
  free(in); free(out);
  cudaFree(d_in); cudaFree(d_out);
  return 0;
}

In [ ]:
%%cu
#include <stdio.h>
#define N  4

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void matrixMulGPU( int * a, int * b, int * c ) {
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] * b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}

//------ Suma ------
__global__ void matrixSumGPU( int * a, int * b, int * c ) {
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] + b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}
//------ Resta ------
__global__ void matrixResGPU( int * a, int * b, int * c ) {
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] - b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}

/*
 * This CPU function already works, and will run to create a solution matrix
 * against which to verify your work building out the matrixMulGPU kernel.
 */

void matrixMulCPU( int * a, int * b, int * c ) {
  int val = 0;
  for( int row = 0; row < N; ++row )
      for( int col = 0; col < N; ++col ) {
        val = 0;
        for ( int k = 0; k < N; ++k )
          val += a[row * N + k] * b[k * N + col];
        c[row * N + col] = val;
      }
}

int main() {
  int *a, *b, *c_cpu, *c_gpu; // Allocate a solution matrix for both the CPU and the GPU operations
  int size = N * N * sizeof (int); // Number of bytes of an N x N matrix

  // Allocate memory
  cudaMallocManaged (&a, size);
  cudaMallocManaged (&b, size);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);

  // Initialize memory; create 2D matrices
  for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col ) {
      a[row*N + col] = row;
      b[row*N + col] = col+2;
      c_cpu[row*N + col] = 0;
      c_gpu[row*N + col] = 0;
    }

/*
   * Assign `threads_per_block` and `number_of_blocks` 2D values
   * that can be used in matrixMulGPU above.
   */

  dim3 threads_per_block(32, 32, 1);
  dim3 number_of_blocks(N / threads_per_block.x + 1, N / threads_per_block.y + 1, 1);
  matrixMulGPU <<< number_of_blocks, threads_per_block >>> ( a, b, c_gpu );
  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");

  // Call the CPU version to check our work
  matrixMulCPU( a, b, c_cpu );

  // Compare the two answers to make sure they are equal
  bool error = false;
  for( int row = 0; row < N && !error; ++row )
    for( int col = 0; col < N && !error; ++col ){
        printf(" [%d][%d] , gpu = %d");

       if (c_cpu[row * N + col] != c_gpu[row * N + col]) {
         printf("FOUND ERROR at c[%d][%d]\n", row, col);
         error = true;
         break;
       }
    }

  if (!error)
    printf("Success!\n");
  
  // Free all our allocated memory
  cudaFree(a); cudaFree(b);
  cudaFree( c_cpu ); cudaFree( c_gpu );
}

 [60178432][60211196] , gpu = 16380 [0][0] , gpu = 0 [0][0] , gpu = 4 [0][0] , gpu = 8 [0][0] , gpu = 12 [0][0] , gpu = 16 [0][0] , gpu = 20 [0][0] , gpu = 24 [0][0] , gpu = 28 [0][0] , gpu = 32 [0][0] , gpu = 36 [0][0] , gpu = 40 [0][0] , gpu = 44 [0][0] , gpu = 48 [0][0] , gpu = 52 [0][0] , gpu = 56 [0][0] , gpu = 60 [0][0] , gpu = 64 [0][0] , gpu = 68 [0][0] , gpu = 72 [0][0] , gpu = 76 [0][0] , gpu = 80 [0][0] , gpu = 84 [0][0] , gpu = 88 [0][0] , gpu = 92 [0][0] , gpu = 96 [0][0] , gpu = 100 [0][0] , gpu = 104 [0][0] , gpu = 108 [0][0] , gpu = 112 [0][0] , gpu = 116 [0][0] , gpu = 120 [0][0] , gpu = 124 [0][0] , gpu = 128 [0][0] , gpu = 132 [0][0] , gpu = 136 [0][0] , gpu = 140 [0][0] , gpu = 144 [0][0] , gpu = 148 [0][0] , gpu = 152 [0][0] , gpu = 156 [0][0] , gpu = 160 [0][0] , gpu = 164 [0][0] , gpu = 168 [0][0] , gpu = 172 [0][0] , gpu = 176 [0][0] , gpu = 180 [0][0] , gpu = 184 [0][0] , gpu = 188 [0][0] , gpu = 192 [0][0] , gpu = 196 [0][0] , gpu = 200 [0][0] , gpu = 204 [0][